In [1]:
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers

from preprocessor import Preprocessor, ImagePreprocessor

2024-04-12 10:37:27.028011: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 10:37:27.028038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 10:37:27.028788: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-12 10:37:27.032979: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_pickle("processed_data/top_500_species_train.pkl")
# ip = ImagePreprocessor(train_ids=data.index)
# images = ip.preprocess()

In [3]:
yy = data.to_numpy()[12:]


In [4]:
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split



XX, YY = data[data.columns[:13]], data[data.columns[13:]]

In [5]:
XX.head(3)

,lon,lat,year,areaInM2,elevation,ALPINE,ATLANTIC,BLACK SEA,BOREAL,CONTINENTAL,MEDITERRANEAN,PANNONIAN,STEPPIC
surveyId,,,,,,,,,,,,,
212,3.099038,43.134956,2021,100.0,26,False,False,False,False,False,True,False,False
222,9.884560,56.912140,2017,79.0,54,False,False,False,False,True,False,False,False
243,8.256020,55.637050,2019,79.0,-2,False,True,False,False,False,False,False,False


In [6]:
YY.head(3)

,51,53,96,129,140,146,167,169,171,249,...,10998,11005,11054,11078,11120,11140,11157,11188,11193,11195
surveyId,,,,,,,,,,,,,,,,,,,,,
212,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
222,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
243,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
X_train, X_val, Y_train, Y_val = train_test_split(XX, YY, test_size=0.2, random_state=0)

In [8]:
model = DecisionTreeRegressor()
model.fit(X_train, Y_train)

DecisionTreeRegressor()

In [9]:
from sklearn.metrics import f1_score

predictions = model.predict(X_val)

f1_score(Y_val.to_numpy(), predictions, average='micro')

0.372686953357142

In [10]:
predictions = pd.DataFrame(predictions, columns=YY.columns)
predictions.head()

,51,53,96,129,140,146,167,169,171,249,...,10998,11005,11054,11078,11120,11140,11157,11188,11193,11195
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
def prepare_submission(data, predictions):
    output_list = []
    for _, row in predictions.iterrows():
        row_string = ""
        for truth, value in zip(row, YY.columns):
            if truth:
                row_string += str(value) + " "
        output_list.append(row_string)

    submission = pd.DataFrame(columns=["surveyId", "predictions"])
    submission["surveyId"] = data.index
    submission["predictions"] = output_list
    return submission


In [12]:
test = pd.read_pickle("processed_data/top_500_species_test.pkl")
test.head()

,lon,lat,year,areaInM2,elevation,ALPINE,ATLANTIC,CONTINENTAL,MEDITERRANEAN
surveyId,,,,,,,,,
642,10.033550,57.12081,2019,707.0,26,False,False,True,False
1792,7.333000,46.22997,2019,10.0,1174,True,False,False,False
3256,1.843658,42.58006,2018,79.0,2070,True,False,False,False
3855,11.720090,46.26149,2021,35.0,1902,True,False,False,False
4889,9.361870,55.90245,2017,79.0,110,False,False,True,False


In [13]:
test[list(set(X_val.columns) - set(test.columns))] = False
test = test[X_val.columns]
# test.head()

In [14]:
results = pd.DataFrame(model.predict(test), columns=YY.columns)
# results.head()

from postprocessor import Postprocessor
pp = Postprocessor(predictions=results, id_keys=list(test.index))

submission_0 = prepare_submission(data=test, predictions=results)
submission_1 = pp.process()

TypeError: __init__() should return None, not 'DataFrame'

In [ ]:
submission.head()
submission.to_csv("decision_tree_500_results.csv", index=False)

In [ ]:
list(YY.columns)
with open('postprocessor-species.txt', 'w') as f:
    for line in list(YY.columns):
        f.write(f"{line}\n")

In [ ]:
import pickle

In [ ]:
with open('postprocess-speices', 'wb') as fp:
    pickle.dump(list(YY.columns), fp)

In [ ]:
with open('postprocess-speices', 'rb') as fp:
    itemlist = pickle.load(fp)

itemlist

[51,
 53,
 96,
 129,
 140,
 146,
 167,
 169,
 171,
 249,
 254,
 262,
 300,
 340,
 391,
 394,
 398,
 423,
 424,
 433,
 469,
 476,
 478,
 509,
 540,
 544,
 559,
 581,
 593,
 623,
 643,
 649,
 651,
 661,
 694,
 786,
 791,
 799,
 822,
 843,
 868,
 890,
 896,
 907,
 958,
 960,
 963,
 976,
 981,
 982,
 1015,
 1018,
 1020,
 1037,
 1038,
 1051,
 1085,
 1086,
 1092,
 1110,
 1131,
 1139,
 1162,
 1170,
 1180,
 1214,
 1254,
 1264,
 1276,
 1288,
 1387,
 1421,
 1424,
 1455,
 1472,
 1497,
 1498,
 1525,
 1531,
 1539,
 1545,
 1654,
 1677,
 1703,
 1705,
 1707,
 1712,
 1715,
 1716,
 1736,
 1757,
 1761,
 1793,
 1818,
 1858,
 1888,
 1889,
 1910,
 1951,
 1964,
 1986,
 2009,
 2013,
 2025,
 2101,
 2109,
 2126,
 2142,
 2145,
 2163,
 2189,
 2211,
 2250,
 2262,
 2308,
 2386,
 2398,
 2421,
 2474,
 2482,
 2490,
 2556,
 2570,
 2587,
 2608,
 2621,
 2634,
 2684,
 2703,
 2747,
 2753,
 2783,
 2788,
 2813,
 2823,
 2840,
 2847,
 2885,
 2891,
 2908,
 2922,
 2949,
 2955,
 2961,
 3020,
 3037,
 3043,
 3067,
 3092,
 3096,
 31